In [1]:
!pip install -U efficientnet

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedShuffleSplit
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.metrics import AUC
from keras import Sequential
import efficientnet.keras as efn

Using TensorFlow backend.


In [ ]:
#Loading in mapping dataframe

meta = pd.read_csv("../input/siim-isic-melanoma-classification/train.csv") 
meta['image_name'] = meta['image_name'] + '.png'

In [42]:
meta.stb.freq(['target'])

,target,Count,Percent,Cumulative Count,Cumulative Percent
0,0,32542,0.98237,32542,0.98237
1,1,584,0.01763,33126,1.00000


In [4]:
# Getting indexes to split data into test and train
sss = StratifiedShuffleSplit(n_splits=2, test_size=.1, random_state=1)
train_index, test_index = sss.split(np.zeros(len(meta)), meta['target'])

# Getting train and validation dataframes
train_df, val_df = meta.iloc[train_index[0], :], meta.iloc[train_index[1], :]

In [19]:
# Creating data generators
train_gen = ImageDataGenerator(horizontal_flip=True,
                               vertical_flip=True,
                               preprocessing_function=efn.preprocess_input)
val_gen = ImageDataGenerator(preprocessing_function=efn.preprocess_input)

# Flowing from dataframes
img_dir = "../input/siic-isic-224x224-images/train"
train_generator = train_gen.flow_from_dataframe(dataframe=train_df, directory=img_dir, 
                                                x_col='image_name',
                                                y_col='target',
                                                class_mode='raw', 
                                                target_size=(224,224),
                                                batch_size=32,
                                                seed=1)
val_generator = val_gen.flow_from_dataframe(dataframe=val_df, directory=img_dir, 
                                                x_col='image_name',
                                                y_col='target',
                                                class_mode='raw', 
                                                target_size=(224,224),
                                                batch_size=32,
                                                seed=1,
                                                shuffle=False)

Found 29813 validated image filenames.
Found 3313 validated image filenames.


In [6]:
# Loading efficientnet model
efficientnet = efn.EfficientNetB4(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

71892992/71892840 [==============================] - 3s 0us/step


In [7]:
# Creating model architecture
model = Sequential()
model.add(efficientnet)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid'))

In [8]:
# Callback and compiling
mcp_save = ModelCheckpoint('model_weights.hdf5', save_best_only=True, monitor='val_auc', mode='max')
model.compile(Adam(lr=.00001), loss='binary_crossentropy', metrics=[AUC(name='auc')])

In [9]:
model.load_weights("../input/flow-weights/model_weights-2.hdf5")

In [ ]:
# Fitting model baby
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
class_weight = {0: 1., 1: 50.}

model.fit_generator(train_generator, validation_data=val_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_steps=STEP_SIZE_VALID, callbacks=[mcp_save], class_weight=class_weight verbose=2, epochs=10)

In [ ]:
# Loading in test labels and images
test_df = pd.read_csv("../input/siim-isic-melanoma-classification/test.csv")
test_img_names = pd.read_csv("../input/siim-isic-melanoma-classification/test.csv")['image_name'].values
test_df['image_name'] = test_df['image_name'] + '.png'

In [ ]:
test_gen = ImageDataGenerator(preprocessing_function=efn.preprocess_input)
test_generator = test_gen.flow_from_dataframe(dataframe=test_df,
                                              x_col='image_name',
                                              directory="../input/siic-isic-224x224-images/test", 
                                                class_mode=None, 
                                                target_size=(224,224),
                                                batch_size=32,
                                                seed=1,
                                                shuffle=False)

In [ ]:
# Making predictions
predictions = model.predict_generator(test_generator).flatten()

In [ ]:
# Submission dataset
sub = pd.DataFrame({"image_name": test_img_names, "target": predictions})

sub.to_csv('flow_sub1.csv', index=False)

In [27]:
## Predicting on data to 
#predictions_eval = model.predict_generator(val_generator).ravel()
#predictions_rounded = np.around(predictions_eval)

# Finding misclassified images
missclassified = predictions_rounded != val_df['target'].values

In [28]:
missed_imgs = val_df[missclassified]

In [32]:
!pip install sidetable

  Created wheel for sidetable: filename=sidetable-0.3.0-py3-none-any.whl size=9391 sha256=0f47b09d3ae12a149ac711b81ebbd6031a5200bae41579e21191b91ca33cfa42
  Stored in directory: /root/.cache/pip/wheels/e3/ce/5e/7e0b8b2f6c714efcbef0db6c3579c5c3c19b842ea360c817c6
Successfully built sidetable


In [33]:
# Simplifying data analysis
import sidetable

In [35]:
missed_imgs.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
17677,ISIC_5391758.png,IP_0093378,male,45.0,lower extremity,melanoma,malignant,1
2722,ISIC_0914168.png,IP_3659172,female,50.0,torso,melanoma,malignant,1
28075,ISIC_8497457.png,IP_2005898,female,70.0,torso,melanoma,malignant,1
32033,ISIC_9677503.png,IP_5086592,male,55.0,head/neck,melanoma,malignant,1
30817,ISIC_9312164.png,IP_4263066,female,65.0,torso,melanoma,malignant,1


In [37]:
missed_imgs.stb.freq(['age_approx'], thresh=.8)

,age_approx,Count,Percent,Cumulative Count,Cumulative Percent
0,45,9,0.155172,9,0.155172
1,75,8,0.137931,17,0.293103
2,70,8,0.137931,25,0.431034
3,55,8,0.137931,33,0.568966
4,65,6,0.103448,39,0.672414
5,60,6,0.103448,45,0.775862
6,Others,13,0.224138,58,1.000000


In [39]:
# I think adding in the meta data could help out.
# Lets actually look at the images
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt    

In [41]:
fig, ax = plt.subplots(nrows=3, ncols=5, figsize=(224, 224))
ax = ax.flatten()

for num, file in enumerate(missed_imgs['image_name']):
    image = load_img(os.path.join(img_dir, file))
    ax[num].imshow(image)
    ax[num].axis('Off')
plt.tight_layout()
plt.axis('Off')
plt.show() 

IndexError: index 15 is out of bounds for axis 0 with size 15